<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [ ]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!pip install flask-cloudflared

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [9]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mega_full' #@param ["Mini", "Mega", "Mega_full"]

# Run the backend web server

In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port,))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py --port {app_port} --model_version {dalle_model} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"your url is: {announce_url}")
    t1.join()

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.trycloudflare.com and run it in the browser.

### Let the fun begin ✨